# Mehul's Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Segmenting and Clustering - Question  1

In [1]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt


In [2]:
import pip

if int(pip.__version__.split('.')[0])>9:
        from pip._internal import main
else:
    from pip import main
def install(package):
    main(['install', package])

install('BeautifulSoup4')

In [3]:
from bs4 import BeautifulSoup as BS

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extract_wiki = requests.get(url).text
wiki_data = BS(extract_wiki, 'lxml')

In [5]:
wiki_data

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"1a5bce74-dba8-49fe-874a-582c656981d4","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":960187814,"wgRevisionId":960187814,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-relat

### Representing HTML as a neat DataFrame

In [6]:
column_name = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_name)

block = wiki_data.find('div', class_='mw-parser-output')
table = block.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1

### Appropriate grouping

In [7]:
tor_df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
tor_df

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn
...,...,...,...
175,M9V\n,Etobicoke\n,"South Steeles, Silverstone, Humbergate, Jamest..."
176,M9W\n,Etobicoke\n,"Northwest, West Humber - Clairville"
177,M9X\n,Not assigned\n,Not assigned\n
178,M9Y\n,Not assigned\n,Not assigned\n


In [8]:
tor_df = tor_df.dropna()
blank = 'Not assigned'
tor_df = tor_df[(tor_df.Postalcode != blank ) & (tor_df.Borough != blank) & (tor_df.Neighborhood != blank)]

In [9]:
def neigh(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
new_tor = tor_df.groupby(['Postalcode', 'Borough'])
tor2_df = new_tor.apply(neigh).reset_index(name='Neighborhood')

In [29]:
tor2_df.head(20)

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn
5,M1H\n,Scarborough\n,Cedarbrae
6,M1J\n,Scarborough\n,Scarborough Village
7,M1K\n,Scarborough\n,"Kennedy Park, Ionview, East Birchmount Park"
8,M1L\n,Scarborough\n,"Golden Mile, Clairlea, Oakridge"
9,M1M\n,Scarborough\n,"Cliffside, Cliffcrest, Scarborough Village West"


In [11]:
tor2_df.to_csv('toronto.csv', index=False)

## Segmenting and Clustering - Question 2

In [16]:
!pip install geocoder

In [17]:
import geocoder

In [18]:
tor_new = pd.read_csv('toronto.csv')
tor_new.head()

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn


In [19]:
def get_ll(postal_code):
    lat_long_coords = None
    while(lat_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_long_coords = g.latlng
    return lat_long_coords
    
get_ll('M4G')

Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M4G%2C+Toronto%2C+Ontario&maxLocations=1


[43.70909000000006, -79.36409999999995]

In [22]:
postal_codes = tor_new['Postalcode']    
coords = [ get_ll(postal_code) for postal_code in postal_codes.tolist() ]

Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M1A%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M1B%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M1C%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M1E%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M1G%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M1H%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M1J%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.a

Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M3Z%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M4A%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M4B%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M4C%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M4E%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M4G%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M4H%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.a

Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M6Y%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M6Z%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M7A%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M7B%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M7C%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M7E%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M7G%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.a

ERROR: Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)


Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M7K%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M7L%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M7M%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M7N%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M7P%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M7R%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find?f=json&text=M7S%0A%2C+Toronto%2C+Ontario&maxLocations=1
Requested https://geocode.a

### Giving new names to the columns

In [23]:
co_ords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
tor_new['Latitude'] = co_ords['Latitude']
tor_new['Longitude'] = co_ords['Longitude']

In [24]:
tor_new[tor_new.Postalcode == 'M5G']

,Postalcode,Borough,Neighborhood,Latitude,Longitude


### Saving to a new CSV

In [25]:
tor_new.to_csv('toronto_2.csv',index=False)

In [28]:
tor_new.head(20)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1A\n,Not assigned\n,Not assigned\n,43.64869,-79.38544
1,M1B\n,Scarborough\n,"Malvern, Rouge",43.81153,-79.19552
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill",43.76575,-79.17520
4,M1G\n,Scarborough\n,Woburn,43.76820,-79.21761
5,M1H\n,Scarborough\n,Cedarbrae,43.76969,-79.23944
6,M1J\n,Scarborough\n,Scarborough Village,43.74309,-79.23526
7,M1K\n,Scarborough\n,"Kennedy Park, Ionview, East Birchmount Park",43.72861,-79.26367
8,M1L\n,Scarborough\n,"Golden Mile, Clairlea, Oakridge",43.71406,-79.28412
9,M1M\n,Scarborough\n,"Cliffside, Cliffcrest, Scarborough Village West",43.72360,-79.23496


## Segmenting and Clustering Neighborhoods in Toronto (part-3)

In [30]:
!pip install geopy
!pip install folium

### Import all important libraries

In [31]:
import folium
import requests 
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [32]:
tor3 = pd.read_csv('toronto_2.csv')

### Finding the coordinates of Toronto

In [35]:
address = 'Toronto, Ontario Canada'
geolocator = Nominatim(user_agent = "msampath21@outlook.com")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto are 43.6534817, -79.3839347.


### Generating Maps to visualise Data 

In [37]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(tor3['Latitude'], tor3['Longitude'], tor3['Borough'], tor3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)
map_toronto

In [38]:
toronto_data = tor3[tor3['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M4E\n,East Toronto\n,The Beaches,43.67703,-79.29542
1,M4K\n,East Toronto\n,"The Danforth West, Riverdale",43.68375,-79.35528
2,M4L\n,East Toronto\n,"India Bazaar, The Beaches West",43.66797,-79.31468
3,M4M\n,East Toronto\n,Studio District,43.66091,-79.33503
4,M4N\n,Central Toronto\n,Lawrence Park,43.72898,-79.39173


In [39]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7,parse_html=False).add_to(map_toronto)  
map_toronto

### Using FourSquare to Explore

In [42]:
CLIENT_ID = '2ELYLWETSOYI5VVU4PXWNCU3B2W4MTGC4UF0PUVZLNIM11XJ'
CLIENT_SECRET = 'A2T1QLDF1CY0K3GBYEGBVNFIME3OVO5CBAJTJNFZCP5LRZ4Q'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2ELYLWETSOYI5VVU4PXWNCU3B2W4MTGC4UF0PUVZLNIM11XJ
CLIENT_SECRET: A2T1QLDF1CY0K3GBYEGBVNFIME3OVO5CBAJTJNFZCP5LRZ4Q


In [44]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        
        results = requests.get(url).json()["response"][0]['items']
        
        venues_list.append([( name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [47]:
exp = toronto_data
toronto_venues = getNearbyVenues(names=exp['Neighborhood'], latitudes=exp['Latitude'],longitudes=exp['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

### List of all nearby places

In [48]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.67703,-79.29542,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.67703,-79.29542,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.67703,-79.29542,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.67703,-79.29542,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant
4,The Beaches,43.67703,-79.29542,Upper Beaches,43.680563,-79.292869,Neighborhood


In [50]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 182 uniques categories.


In [51]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,History Museum,Home Service,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Office,Opera House,Organic Grocery,Park,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [52]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [53]:
num_top_venues = 5
for neigh in toronto_grouped['Neighborhood']:
    print("----"+neigh+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0        Cocktail Bar  0.07
1  Seafood Restaurant  0.07
2            Beer Bar  0.07
3        Concert Hall  0.03
4              Bistro  0.03


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0  Furniture / Home Store  0.07
1               Gift Shop  0.07
2             Coffee Shop  0.07
3             Supermarket  0.07
4                    Café  0.07


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
          venue  freq
0          Café  0.10
1         Hotel  0.07
2    Steakhouse  0.07
3   Coffee Shop  0.07
4  Concert Hall  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                  venue  freq
0    Italian Restaurant  0.10
1  Gym / Fitness Center  0.07
2           Coffee Shop  0.07
3           Yoga Studio  0.03
4   Peruvian Restaurant  0.03


----Central Bay Street----
            

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [55]:
import numpy as np
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(39, 11)

In [56]:
from sklearn.cluster import KMeans
import sklearn.cluster.k_means_
km = KMeans(n_clusters=3, init='k-means++', max_iter=100, n_init=1, 
  verbose=True)

In [57]:
kclusters = 10
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)
print(kmeans.labels_[0:10])
print(len(kmeans.labels_))

[1 1 1 1 1 1 1 1 1 1]
39


In [58]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E\n,East Toronto\n,The Beaches,43.67703,-79.29542,1,College Gym,Trail,Health Food Store,Asian Restaurant,Pub,Wine Shop,Discount Store,Falafel Restaurant,Event Space,Electronics Store
1,M4K\n,East Toronto\n,"The Danforth West, Riverdale",43.68375,-79.35528,8,Grocery Store,Coffee Shop,Bus Line,Intersection,Discount Store,Ice Cream Shop,Park,Deli / Bodega,Falafel Restaurant,Event Space
2,M4L\n,East Toronto\n,"India Bazaar, The Beaches West",43.66797,-79.31468,1,Sandwich Place,Pizza Place,Fast Food Restaurant,Park,Sushi Restaurant,Pet Store,Burrito Place,Pub,Liquor Store,Movie Theater
3,M4M\n,East Toronto\n,Studio District,43.66091,-79.33503,1,Brewery,Italian Restaurant,Pet Store,Arts & Crafts Store,Coffee Shop,Diner,Bar,Latin American Restaurant,Pizza Place,Sandwich Place
4,M4N\n,Central Toronto\n,Lawrence Park,43.72898,-79.39173,3,Park,Home Service,Wine Shop,Diner,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Donut Shop,Dog Run


### Final visualisation

In [59]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
map_clusters